# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print("Current directory: ",os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    print("Filepath of oroginal csv: ",root)
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print("Event csv data files:")
    for file in file_path_list:
        print(file)

Current directory:  /home/workspace
Filepath of oroginal csv:  /home/workspace/event_data
Event csv data files:
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-02-events.csv
/home/workspace/event_data/2018-11-03-events.csv
/home/workspace/event_data/2018-11-13-events.csv
/home/workspace/event_data/2018-11-14-events.csv
/home/workspace/event_data/2018-11-10-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-30-events.csv
/home/workspace/event_data/2018-11-17-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-12-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-08-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-28-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

#drop .ipynb_checkpoints
#file_path_list = [ x for x in file_path_list if ".ipynb_checkpoints" not in x]

# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        next(csvreader)
         # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            if line[0] == "artist":
                continue
            full_data_rows_list.append(line) 
            
# total number of rows 
print(len(full_data_rows_list))

# creating event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_full.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_full.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create tables for Sparkify app

## CSV file  <font color=red>event_datafile_full.csv</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_full.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Connection to a Apache Cassandra

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
query = "create keyspace if not exists ml with replication"
params = "= {'class':'SimpleStrategy', 'replication_factor':1}; "
try:
    session.execute(query+params)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('ml')
except Exception as e:
    print(e)

## Create tables

#### Table **song_library**
For the first query we create table contains the following columns:
* <font color=red>artist</font>- <font color=blue>format **text**</font>  (*artist name*)
* <font color=red>title</font> - <font color=blue>format **text**</font> (*song title*)
* <font color=red>length</font> - <font color=blue>format **float**</font> (*song length*)
* <font color=red>session_id</font> - <font color=blue>format **int**</font> (*identificator of session*)
* <font color=red>item_in_session</font> - <font color=blue>format **int**</font> (*item in session_id*)

<font color=blue>**Primary key**</font> consists of session_id and item_in_session because these columns uniquely identify records of the client session <br />
In our case <font color=red>session_id</font> - is a **partiotion key** and <font color=red>item_in_session</font> is a **clustering key** <br />
This table will be useful for queries about all information on the listened content for a every user session's and items.

In [8]:
query = "create table if not exists song_in_session"
query = query + "(artist text, title text, length float, session_id int, item_in_session int, \
                    primary key(session_id,item_in_session));"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Table **user_session**
For the second query we create table contains the following columns:
* <font color=red>artist</font>- <font color=blue>format **text**</font>  (*artist name*)
* <font color=red>title</font> - <font color=blue>format **text**</font> (*song title*)
* <font color=red>user_firstname</font> - <font color=blue>format **text**</font> (*user firstname*)
* <font color=red>user_lastname</font> - <font color=blue>format **text**</font> (*user lastname*)
* <font color=red>user_id</font> - <font color=blue>format **int**</font> (*user identification*)
* <font color=red>session_id</font> - <font color=blue>format **int**</font> (*session identificator*)
* <font color=red>item_in_session</font> - <font color=blue>format **text**</font> (*item in session_id*)

<font color=blue>**Primary key**</font> consists of <font color=red>user_id</font> and <font color=red>session_id</font> because these columns uniquely identify records of the user session <br />
In our case <font color=red>user_id</font> and <font color=red>session_id</font> - is a **partiotion keys** and <font color=red>item_in_session</font> is a **clustering key**, it's needed for sorting songs  <br />
This table will be useful for queries about user information and his used content for a every user session's and items.

In [9]:
query = "create table if not exists user_session"
query = query + "(artist text, title text, user_firstname text, user_lastname text, user_id int, session_id int, item_in_session int, \
                    primary key((user_id, session_id),item_in_session));"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Table **user_songs**

For the third query we create table contains the following columns:
* <font color=red>user_id</font>- <font color=blue>format **int**</font>  (*artist name*)
* <font color=red>user_firstname</font> - <font color=blue>format **text**</font> (*song title*)
* <font color=red>user_lastname</font> - <font color=blue>format **text**</font> (*song length*)
* <font color=red>title</font> - <font color=blue>format **text**</font> (*identificator of session*)

<font color=blue>**Primary key**</font> consists of <font color=red>title</font> and <font color=red>user_id</font> because these columns uniquely identify records of the client activity (which music listened by user)<br />
In our case <font color=red>title</font> - is a **partiotion key** and <font color=red>user_id</font> is a **clustering key** <br />
This table will be useful for queries about all information on the listened content for a every user at all

In [10]:
query = "create table if not exists user_songs"
query = query + "(title text, user_id int, user_firstname text, user_lastname text, \
                    primary key(title,user_id));"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Create queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [11]:
# We have provided part of the code to set up the CSV file.
file = 'event_datafile_full.csv'

# INSERT statements from event_datafile_full.csv into song_in_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_in_session (artist, title, length, session_id, item_in_session)"
        query = query + "values (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [12]:
## SELECT top 5 statement to verify the data was entered into the table
query = "select artist, title, length, session_id, item_in_session from song_in_session limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
        print (row.artist, " | ", row.title, " | ", row.length, " | ", row.session_id, " | ", row.item_in_session)

Regina Spektor  |  The Calculation (Album Version)  |  191.08526611328125  |  23  |  0
Octopus Project  |  All Of The Champs That Ever Lived  |  250.95791625976562  |  23  |  1
Tegan And Sara  |  So Jealous  |  180.06158447265625  |  23  |  2
Dragonette  |  Okay Dolores  |  153.39056396484375  |  23  |  3
Lil Wayne / Eminem  |  Drop The World  |  229.58975219726562  |  23  |  4


In [13]:
# query #1
query = "select artist, title, length from song_in_session where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, " | " , row.title, " | ", row.length)

Faithless  |  Music Matters (Mark Knight Dub)  |  495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [14]:
## We have provided part of the code to set up the CSV file.
file = 'event_datafile_full.csv'

# INSERT statements from event_datafile_full.csv into user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_session (artist, title, user_firstname, user_lastname, user_id, session_id, item_in_session)"
        query = query + "values (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))                   

In [15]:
# SELECT top 5 statement to verify the data was entered into the table
query = "select artist, title, user_firstname, user_lastname, user_id, session_id, item_in_session from user_session limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
        print (row.artist, " | ", row.title, " | ", row.user_firstname, " | ", row.user_lastname, " | ", row.user_id,\
              " | ", row.session_id, " | ", row.item_in_session, " | ")

System of a Down  |  Sad Statue  |  Emily  |  Benson  |  58  |  768  |  0  | 
Ghostland Observatory  |  Stranger Lover  |  Emily  |  Benson  |  58  |  768  |  1  | 
Evergreen Terrace  |  Zero  |  Emily  |  Benson  |  58  |  768  |  2  | 
Deftones  |  Head Up (LP Version)  |  Kinsley  |  Young  |  85  |  776  |  2  | 
The Notorious B.I.G.  |  Playa Hater (Amended Version)  |  Kinsley  |  Young  |  85  |  776  |  3  | 


In [16]:
# query #2
query = "select artist, title, user_firstname, user_lastname from user_session \
            where user_id = 10 and session_id = 182"
try:
    session.execute(query)
except Exception as e:
    print(e)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [17]:
## We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_full.csv'

# INSERT statements from event_datafile_full.csv into user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_songs (title, user_id, user_firstname, user_lastname)"
        query = query + "values (%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [18]:
# SELECT top 5 statement to verify the data was entered into the table
query = "select user_id, user_firstname, user_lastname, title from user_songs limit 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
        print (row.user_id, " | ", row.user_firstname, " | ", row.user_lastname, " | ", row.title)

49  |  Chloe  |  Cuevas  |  Wonder What's Next
49  |  Chloe  |  Cuevas  |  In The Dragon's Den
44  |  Aleena  |  Kirby  |  Too Tough (1994 Digital Remaster)
49  |  Chloe  |  Cuevas  |  Rio De Janeiro Blue (Album Version)
15  |  Lily  |  Koch  |  My Place


In [19]:
# query #3
query = "select user_firstname, user_lastname from user_songs where title = 'All Hands Against His Own'"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Drop the tables before closing out the sessions

In [20]:
tables_list = ["song_in_session","user_session","user_songs"]
for tab in tables_list:
    query = "drop table "+tab
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()